In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
glove_wiki_vectors.most_similar('computer')

[('computers', 0.875198245048523),
 ('software', 0.8373122215270996),
 ('technology', 0.7642159461975098),
 ('pc', 0.7366448640823364),
 ('hardware', 0.7290391325950623),
 ('internet', 0.72867751121521),
 ('desktop', 0.7234441637992859),
 ('electronic', 0.7221828699111938),
 ('systems', 0.7197922468185425),
 ('computing', 0.7141730189323425)]

In [7]:
glove_wiki_vectors.most_similar('language')

[('languages', 0.8260654807090759),
 ('word', 0.7464082837104797),
 ('spoken', 0.7381492853164673),
 ('arabic', 0.7318817973136902),
 ('english', 0.7214903831481934),
 ('dialect', 0.6912704110145569),
 ('vocabulary', 0.6908208131790161),
 ('text', 0.685594916343689),
 ('translation', 0.6810673475265503),
 ('words', 0.6715823411941528)]

In [8]:
glove_wiki_vectors.most_similar('tara') 

[('lipinski', 0.6272903680801392),
 ('ruby', 0.6021775007247925),
 ('rani', 0.5890056490898132),
 ('cathy', 0.5828190445899963),
 ('veronica', 0.5802205801010132),
 ('sonya', 0.5787814855575562),
 ('carrie', 0.5721156001091003),
 ('kristen', 0.5680761337280273),
 ('duggan', 0.5659471750259399),
 ('heather', 0.564447283744812)]

In [9]:
glove_wiki_vectors.most_similar('statistics') 

[('figures', 0.7243505716323853),
 ('report', 0.6801027655601501),
 ('data', 0.6705881953239441),
 ('statistic', 0.654253363609314),
 ('bureau', 0.6409845352172852),
 ('according', 0.64072185754776),
 ('estimates', 0.6405200958251953),
 ('statistical', 0.6386309266090393),
 ('survey', 0.6346055269241333),
 ('gdp', 0.6314312815666199)]

In [10]:
glove_wiki_vectors.most_similar('what')

[('how', 0.9303213953971863),
 ('why', 0.9196362495422363),
 ('fact', 0.9069437980651855),
 ('know', 0.8876389861106873),
 ('that', 0.8810365796089172),
 ('think', 0.8772969841957092),
 ('so', 0.875309944152832),
 ('even', 0.8751895427703857),
 ('something', 0.874744176864624),
 ('if', 0.8702542781829834)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
wp1_similarity = glove_wiki_vectors.similarity("coast", "shore")
wp2_similarity = glove_wiki_vectors.similarity("clothes", "closet")
wp3_similarity = glove_wiki_vectors.similarity("old", "new")
wp4_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
wp5_similarity = glove_wiki_vectors.similarity("dog", "cat")
wp6_similarity = glove_wiki_vectors.similarity("tree", "lawyer")

print("Cosine similarity of coast and shore = %0.3f" % wp1_similarity)
print("Cosine similarity of clothes and closet = %0.3f" % wp2_similarity)
print("Cosine similarity of old and new = %0.3f" % wp3_similarity)
print("Cosine similarity of smart and intelligent = %0.3f" % wp4_similarity)
print("Cosine similarity of dog and cat = %0.3f" % wp5_similarity)
print("Cosine similarity of tree and lawyer = %0.3f" % wp6_similarity)

Cosine similarity of coast and shore = 0.700
Cosine similarity of clothes and closet = 0.546
Cosine similarity of old and new = 0.643
Cosine similarity of smart and intelligent = 0.755
Cosine similarity of dog and cat = 0.880
Cosine similarity of tree and lawyer = 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    if word in glove_wiki_vectors:
        print("The word %s is in the vocabulary" % word)
    else:
        print("The word %s is not in the vocabulary" % word)

The word covididiot is not in the vocabulary
The word fomo is not in the vocabulary
The word frenemies is in the vocabulary
The word anthropause is not in the vocabulary
The word photobomb is not in the vocabulary
The word selfie is not in the vocabulary
The word pxg is not in the vocabulary
The word pacg is not in the vocabulary
The word cct is in the vocabulary
The word escc is in the vocabulary


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.1428324

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
analogy("woman", "secretary", "man")

woman : secretary :: man : ?


,Analogy word,Score
0,general,0.736190
1,chief,0.692401
2,deputy,0.671780
3,chairman,0.664504
4,rumsfeld,0.653890
5,donald,0.635758
6,spokesman,0.628681
7,vice,0.618626
8,president,0.618278
9,minister,0.616832


In [20]:
glove_wiki_vectors.similarity("woman", "intelligence"), glove_wiki_vectors.similarity("man", "intelligence")

(0.24037766, 0.3724517)

In [21]:
glove_wiki_vectors.similarity("indigenous", "money"), glove_wiki_vectors.similarity("white", "money")

(0.12011738, 0.41533148)

In [22]:
glove_wiki_vectors.similarity("woman", "education"), glove_wiki_vectors.similarity("man", "education")

(0.34139162, 0.2877106)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. Looking at my observations in 1.4, we can see that these embeddings indicate various societal stereotypical biases:
   * the analogy "woman : secretary :: man : ?" reveals systemic gender bias by connecting men with leadership roles like "chief" and "president" and women with administrative roles like "secretary". Similarly, the similarity score "woman" vs. "man" with "intelligence" (0.240 vs. 0.372) shows embeddings' stronger association of men with intelligence, reinforcing once again gender stereotypes. However, we could explain some of this bias with the word frequency in the training data, and potentially the age of the data, but these biases persist in the embeddings.
   * on the other hand, the similarity scores "indigenous" vs. "white" with "money" (0.120 vs. 0.415) suggest a stronger association of "white" with money, reflecting economic or social disparities in the data.
   * surprisingly, "woman" has a slightly stronger association with "education" than "man" (0.341 vs. 0.288), which may reflect societal narratives about women and education. However, this can oversimplify roles and contexts, ignoring men's contributions in education and reinforcing stereotypes.
2. An example of how using these embeddings with biases could cause harm in the real world is directly if it was used in evaluating applicants for job hiring. These embeddings would indicate that all applicants who identify as "woman" are automatically associated with the lower leadership positions, intelligence, and education levels, when compared to the male applicants.This might influence hiring manager's decision who to hire - which would automatically reinforce decreased employee and company diversity, representation of women in certain fields, limit opportunities for women-identifying individuals, and this would further impact company's results, outcomes, and so much more, further entrenching societal stereotypes in professional environments.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [26]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [27]:
def preprocess_spacy(doc, min_token_len = 2, not_tags = ["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"]):  
    cleaned_up = []
    for token in doc:
        if token.like_url: cleaned_up.append("URL")
        elif token.like_email: cleaned_up.append("EMAiL")
        elif token.like_num: cleaned_up.append("NUM")
        elif (
            token.pos_ not in not_tags
            and len(token) > min_token_len
            and token.is_stop == False
        ):
            cleaned_up.append(token.lemma_.lower())
    return " ".join(cleaned_up)

In [28]:
import re
def preprocess_text(text):
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"""[\n\r]+""", "", text)
    text = re.sub(r"""[\*\~]+""", "", text)
    text = re.sub(r"""[\/]+""", " ", text)

    return text

In [29]:
df["text"] = df["text"].apply(preprocess_text)
df.dropna()

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,It's not a bad question: I don't have any ref...,1,comp.graphics
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics
3,"The following problem is really bugging me, an...",2,comp.windows.x
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone :: I am looking for some software ...,1,comp.graphics
4559,Archive-name: x-faq part3 Last-modified: 1993 ...,2,comp.windows.x
4560,"That's nice, but it doesn't answer the questi...",6,talk.politics.guns
4561,"Hi, I just got myself a Gateway 4DX-33V and tr...",2,comp.windows.x


In [30]:
# creating col with preprocessed text
df["text_pp"] = [preprocess_spacy(spacy_text) for spacy_text in nlp.pipe(df["text"].tolist())] 

In [31]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me, an...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi, I'd like to subscribe to Leadership Magazi...",5,soc.religion.christian,like subscribe leadership magazine wonder NUM ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

The preprocessing steps we decided to do include:
* removing slashes, newlines, and other formatting characters in order to standardize the text
* sentence segmentation and tokenization for easier analysis of the text
* replacing URLs with the token "URL", emails with "EMAIL", and numbers with "NUM" since these tokens are often unique, and therefore don't add meaningful information for topic modeling
*  filtering out certain tokens based on part-of-speech tags such as adverbs, pronouns, etc., since they are not useful for topic modeling
* removing whitespaces, newlines, return characters, special characters, slashes since they are irrelevant for text analysis
* applying lemmatization to normalize words to their base forms


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(df["text_pp"])

X_vectorizer = vectorizer.transform(df["text_pp"])

In [33]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(n_components = n_topics, learning_method = "batch", max_iter = 10, random_state = 42)
document_topics = lda.fit_transform(X_vectorizer)

The reason behind picking 4 topics was based on the categories initially chosen - cats = ["rec.sport.hockey", "rec.sport.baseball", "soc.religion.christian", "alt.atheism", "comp.graphics", "comp.windows.x", "talk.politics.mideast", "talk.politics.guns"]. From these categories, we could recognize 4 distinguishable topics - sports, computer, religion, and politics.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [34]:
sorting = np.argsort(lda.components_, axis = 1)[:, ::-1]
feature_names = np.array(vectorizer.get_feature_names_out())

In [36]:
import mglearn

mglearn.tools.print_topics(
    topics = range(3),
    feature_names = feature_names,
    sorting = sorting,
    topics_per_chunk = 5,
    n_words = 10,
)

topic 0       topic 1       topic 2       
--------      --------      --------      
num           jews          num           
game          greek         people        
team          turkish       god           
year          jewish        say           
play          greece        know          
player        turkey        think         
win           palestine     time          
good          num           come          
season        israel        believe       
league        know          go            




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [117]:
for i in range(5):
    print(df.iloc[i]['text'])
    print("\n")
    print("Document Topic Assignment: ", document_topics[i]) 
    print("\n")

You know, I was reading 18 U.S.C. 922 and something just did not make sence and I was wondering if someone could help me out. Say U.S.C. 922 : (1) Except as provided in paragraph (2), it shall be unlawful for any person to transfer or possess a machinegun. Well I got to looking in my law dictionary and I found that a "person" might also be an artificial entity that is created by government and has no rights under the federal constitution. So, what I don't understand is how a statute like 922 can be enforced on an individual. So someone tell me how my government can tell me what I can or cannot possess. Just passing a law does not make it LAW. Everyone knows that laws are constitional until it goes to court. So, has it ever gone to court, not just your run of the mill "Ok I had it I am guilty, put me in jail" Has anyone ever claimed that they had a right to possess and was told by the Supreme Court that they didn't have that right? 


Document Topic Assignment:  [0.06698166 0.00325128 0

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)